In [ ]:
# When comparing two data sets using statistical tools, it is not
# sufficient to simply compare their average or median values. This is because
# such results can be skewed by randomness. For example, suppose we are
# calculating the median values of ten seeds. If one algorithm gets "lucky"
# and happens to use more above-average seeds, the estimated median will
# be skewed. Therefore, it is necessary to check the statistical significance of
# results. This is exactly what the indifferent entries are displaying. To determine
# statistical significance, the MOEA Framework uses the Kruskal-Wallis
# and Mann-Whitney U tests with 95% confidence intervals.

In [4]:
from scipy import stats
import sys
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]
sys.path.append('..')
import rbf_functions

In [5]:
rbfs = [rbf_functions.original_rbf,
        rbf_functions.squared_exponential_rbf,
        rbf_functions.inverse_quadratic_rbf,
        rbf_functions.inverse_multiquadric_rbf,
        rbf_functions.exponential_rbf,
        rbf_functions.matern32_rbf,
        rbf_functions.matern52_rbf,
       ]

# Mann-Whitney local refset

In [15]:
gddata = {}
eidata = {}
hvdata = {}
varlist = ['10', '20', '30', '40', '50', '60', '70', '80', '90', '100']
for entry in rbfs:
    rbf = entry.__name__
#     df_gd_data = pd.read_csv(f"gd_all/{rbf}_gd_all.csv")
#     df_ei_data = pd.read_csv(f"ei_all/{rbf}_ei_all.csv")
    df_hv_data = pd.read_csv(f"hv/{rbf}_hv.csv")
    gddata[rbf] = {}
    eidata[rbf] = {}
    hvdata[rbf] = {}
    for i in varlist: # add max hv value at 100.000 nfe to dict for each seed
#         gddata[rbf][i] = df_gd_data[i].dropna().iloc[-1] 
#         eidata[rbf][i] = df_ei_data[i].dropna().iloc[-1]
        hvdata[rbf][i] = df_hv_data[i].dropna().iloc[-1]
# df_gd = pd.DataFrame.from_dict(gddata, orient='index')
# df_ei = pd.DataFrame.from_dict(eidata, orient='index')
df_hv = pd.DataFrame.from_dict(hvdata, orient='index')

In [21]:
# Max attained hypervolume per rbf for each seed
df_hv

,10,20,30,40,50,60,70,80,90,100
original_rbf,0.465507,0.392343,0.476701,0.523039,0.426644,0.428990,0.439437,0.501185,0.491427,0.515935
squared_exponential_rbf,0.380046,0.410288,0.322934,0.467233,0.435752,0.422746,0.537120,0.464505,0.352284,0.503771
inverse_quadratic_rbf,0.393558,0.362230,0.367310,0.401038,0.368133,0.361958,0.389456,0.368153,0.356869,0.392215
inverse_multiquadric_rbf,0.483241,0.454009,0.469192,0.472602,0.480215,0.480522,0.474922,0.471801,0.477761,0.464382
exponential_rbf,0.226676,0.316662,0.344283,0.351767,0.235409,0.356604,0.242845,0.340627,0.222944,0.275830
matern32_rbf,0.400585,0.432749,0.462341,0.329929,0.266519,0.428962,0.448612,0.281948,0.401901,0.498354
matern52_rbf,0.326209,0.324530,0.445316,0.370571,0.284946,0.363744,0.332511,0.425792,0.275986,0.426712


In [ ]:
# method:
# 'asymptotic': compares the standardized test statistic against the normal distribution, correcting for ties.
# 'exact': computes the exact p-value by comparing the observed  statistic against the exact distribution of the  statistic under the null hypothesis. No correction is made for ties.
# 'auto': chooses 'exact' when the size of one of the samples is less than 8 and there are no ties; chooses 'asymptotic' otherwise.

In [17]:
df_mannwhitney = pd.DataFrame(columns=df_hv.index)
df_pval = pd.DataFrame(columns=df_hv.index)
for i, row in df_hv.iterrows():
    for j, row2 in df_hv.iterrows():
        mannwhit, pval = stats.mannwhitneyu(row, row2, method='auto')
        df_mannwhitney.loc[i, j] = mannwhit
        df_pval.loc[i, j] = pval

In [18]:
df_mannwhitney

,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,50.0,68.0,98.0,48.0,100.0,78.0,93.0
squared_exponential_rbf,32.0,50.0,76.0,24.0,95.0,63.0,77.0
inverse_quadratic_rbf,2.0,24.0,50.0,0.0,100.0,31.0,61.0
inverse_multiquadric_rbf,52.0,76.0,100.0,50.0,100.0,89.0,100.0
exponential_rbf,0.0,5.0,0.0,0.0,50.0,15.0,22.0
matern32_rbf,22.0,37.0,69.0,11.0,85.0,50.0,67.0
matern52_rbf,7.0,23.0,39.0,0.0,78.0,33.0,50.0


In [20]:
df_pval

,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,1.0,0.185877,0.00033,0.909722,0.000183,0.037635,0.001315
squared_exponential_rbf,0.185877,1.0,0.053903,0.053903,0.000769,0.344704,0.045155
inverse_quadratic_rbf,0.00033,0.053903,1.0,0.000183,0.000183,0.161972,0.427355
inverse_multiquadric_rbf,0.909722,0.053903,0.000183,1.0,0.000183,0.003611,0.000183
exponential_rbf,0.000183,0.000769,0.000183,0.000183,1.0,0.009108,0.037635
matern32_rbf,0.037635,0.344704,0.161972,0.003611,0.009108,1.0,0.212294
matern52_rbf,0.001315,0.045155,0.427355,0.000183,0.037635,0.212294,1.0
